In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("KeyAggs").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

21/12/03 15:11:59 WARN Utils: Your hostname, duyvan-Inspiron-5402 resolves to a loopback address: 127.0.1.1; using 10.0.130.75 instead (on interface wlp44s0)
21/12/03 15:11:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/03 15:12:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/03 15:12:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
sc = spark.sparkContext

In [6]:
# rdd = sc.parallelize([1,2,3])
# rdd1 = rdd.distinct()
# print(rdd1.toDebugString())

In [7]:
ord = sc.textFile('orders')
ordItems = sc.textFile('order_items')

In [15]:
ordGrp = ordItems.map(lambda x : (int(x.split(',')[2]), float(x.split(',')[4]) ) )

In [19]:
ordGrp1 = ordGrp.groupByKey()

In [21]:
ordGrp1.mapValues(sum).take(5)

[(502, 3147800.0),
 (1014, 2888993.939999287),
 (926, 14870.69999999995),
 (134, 20025.0),
 (276, 29398.810000000012)]

In [22]:
ordGrp1.map(lambda x : (x[0], sum(x[1]))).take(5)

[(502, 3147800.0),
 (1014, 2888993.939999287),
 (926, 14870.69999999995),
 (134, 20025.0),
 (276, 29398.810000000012)]

In [23]:
ordGrp2 = ordItems.map(lambda x : (int(x.split(',')[1]), float(x.split(',')[4]) ) )

In [24]:
from operator import add

In [25]:
ordGrp3 = ordGrp2.reduceByKey(add)

In [26]:
ordGrp3.take(5)

[(2, 579.98),
 (4, 699.85),
 (8, 729.8399999999999),
 (10, 651.9200000000001),
 (12, 1299.8700000000001)]

In [28]:
ordGrp4 = ordGrp2.reduceByKey(lambda x,y : x+y)

In [29]:
ordGrp4.take(5)

[(2, 579.98),
 (4, 699.85),
 (8, 729.8399999999999),
 (10, 651.9200000000001),
 (12, 1299.8700000000001)]

In [1]:
data = [(2,"jose",200),(2,"jim",250),(2,"tina",130),(4,"jim",50),(4,"tina",300),\
    (4,"jose",150),(4,"ram",200),(7,"tina",200),(7,"jose",300),(7,"jim",80)]

In [4]:
ordItems = sc.parallelize(data, 2)

In [5]:
ordPair = ordItems.map(lambda x : (x[0], (x[1], x[2]) ))

In [6]:
zero_val = 0

def seq_op(acc, elem):
    if (acc > elem[1]):
        return acc
    else: return elem[1]

def comb_op(acc1, acc2):
    if (acc1 > acc2):
        return acc1
    else:
        return acc2

In [7]:
agg_orderItems = ordPair.aggregateByKey(zero_val, seq_op, comb_op)

In [8]:
agg_orderItems.take(5)

[(2, 250), (4, 300), (7, 300)]

In [9]:
zero_val = ('',0)

def seq_op(acc, elem):
    if (acc[1] > elem[1]):
        return acc
    else: return elem

def comb_op(acc1, acc2):
    if (acc1[1] > acc2[1]):
        return acc1
    else:
        return acc2

In [10]:
agg_orderItems1 = ordPair.aggregateByKey(zero_val, seq_op, comb_op)

In [11]:
agg_orderItems1.take(5)

[(2, ('jim', 250)), (4, ('tina', 300)), (7, ('jose', 300))]

In [12]:
zero_val = (0,0)

def seq_op(acc, elem):
    return (acc[0] + elem[1], acc[1] + 1)

def comb_op(acc1, acc2):
    return (acc1[0] + acc2[0], acc1[1] + acc2[1] )

In [13]:
agg_orderItems2 = ordPair.aggregateByKey(zero_val, seq_op, comb_op)

In [14]:
agg_orderItems2.take(5)

[(2, (580, 3)), (4, (700, 4)), (7, (580, 3))]

In [4]:
# Count by Key
ord = sc.textFile('orders')

In [6]:
ordKV = ord.map(lambda x : (x.split(',')[3], 1))

In [7]:
ordKV.take(5)

[('CLOSED', 1),
 ('PENDING_PAYMENT', 1),
 ('COMPLETE', 1),
 ('CLOSED', 1),
 ('COMPLETE', 1)]

In [8]:
countByStatus = ordKV.countByKey()

In [9]:
countByStatus.items()

dict_items([('CLOSED', 7556), ('PENDING_PAYMENT', 15030), ('COMPLETE', 22899), ('PROCESSING', 8275), ('PAYMENT_REVIEW', 729), ('PENDING', 7610), ('ON_HOLD', 3798), ('CANCELED', 1428), ('SUSPECTED_FRAUD', 1558)])

In [10]:
spark.stop()